In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install git+https://github.com/fastai/fastai2 
from fastai2.vision.all import *

In [ ]:
# train and test csv
train = pd.read_csv("/kaggle/input/shopee-code-league-2020-product-detection/train.csv")
test = pd.read_csv("/kaggle/input/shopee-code-league-2020-product-detection/test.csv")
# paths leading to images
train_path = Path("/kaggle/input/shopee-code-league-2020-product-detection/resized/train/")
test_path = Path("/kaggle/input/shopee-code-league-2020-product-detection/resized/test/")

In [ ]:
# add the category to filename for easier usage with fastai API
train['filename'] = train.apply(lambda x: str(x.category).zfill(2) + '/' + x.filename, axis=1)
train

In [ ]:
# train in a 10% subset of the data
# to speed up experimentation
# comment these lines out to increase accuracy (but necessitates longer training time)
#from sklearn.model_selection import train_test_split
#_, train = train_test_split(train, test_size=0.05, stratify=train.category)

In [ ]:
#aug_transforms??

In [ ]:
def get_dls_from_df(df,size,bs):
    
    # Hyper parameters for Data Preprocessing
    batchsize = bs
    item_tfms = [RandomResizedCrop(size+32, min_scale=0.75)]
    batch_tfms = [*aug_transforms(size=size,do_flip=False,flip_vert = True,max_rotate=15,max_zoom=1.3), Normalize.from_stats(*imagenet_stats)]
    #
    
    df = df.copy()
    options = {
        "item_tfms": item_tfms,
        "batch_tfms": batch_tfms,
        "bs": batchsize,
    }
    dls = ImageDataLoaders.from_df(df, train_path, **options)
    return dls

In [ ]:
defaults.device = torch.device('cuda') # makes sure the gpu is used

# Train

In [ ]:
### First
dls = get_dls_from_df(train,128, 32)
learn = cnn_learner(dls, resnet50, metrics=accuracy)
# Fit
lr = 0.008
learn.fit_one_cycle(5, slice(lr))
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-4, lr/10))
# Save model 
learn.export(Path("/kaggle/working/export.pkl"))
learn.model_dir='/kaggle/working/'
learn.save('model')
# Delete
del learn
torch.cuda.empty_cache()

### Final
dls = get_dls_from_df(train,244, 16)
learn = cnn_learner(dls, resnet50, metrics=accuracy)
# Load Model
learn.model_dir='/kaggle/working/'
learn.load('model')
# Fit
lr = 1e-3/2
learn.fit_one_cycle(5, slice(lr))
learn.unfreeze()
learn.fit_one_cycle(8, slice(1e-4, lr/10))
# Save model 
learn.model_dir='/kaggle/working/'
learn.save('model_final')


# Test Prediction

In [ ]:
test_images = test.filename.apply(lambda fn: test_path/fn)
test_dl = dls.test_dl(test_images)

In [ ]:
preds = learn.get_preds(dl=test_dl, with_decoded=True)
#save raw predictions
torch.save(preds, "rawpreds")

# **Submission**

In [ ]:
submission  = test[["filename"]]
submission["category"] = preds[2]
submission["category"] = submission.category.apply(lambda c: str(c).zfill(2))
submission

In [ ]:
#save the submissions as CSV
submission.to_csv("submission.csv")

In [ ]:
learn.export(Path("/kaggle/working/export.pkl"))